- Building HMM to perform speech recognition.

In [3]:
import os
import argparse

import numpy as np 
from scipy.io import wavfile
from hmmlearn import hmm
from python_speech_features import mfcc

In [2]:
# function to parse input arguments
def build_arg_parser():
    parser = argparse.ArgumentParser(
        description="Trains the HMM Classifier"
    )
    parser.add_argument(
        "--input-folder",
        dest="input_folder",
        required=True,
        help="Input folder containing the audio files in subfolders"
    )
    return parser

In [4]:
# define a class to model HMMs

class HMMTrainer(object):
    def __init__(self,model_name='GaussianHMM', n_components=4, cov_type='diag', n_iter=1000):
        self.model_name = model_name
        self.n_components = n_components
        self.cov_type = cov_type
        self.n_iter = n_iter
        self.models = [] # where we are going to store all the models for words we want to identify

        if self.model_name == 'GaussianHMM':
            self.model = hmm.GaussianHMM(
                n_components=self.n_components,
                covariance_type=self.cov_type,
                n_iter=self.n_iter
                )
        else:
            raise TypeError('Invalid Model Type')

    # X is a 2D numpy array where each row is 13D : look into this one keenly.
    def train(self, X):
        np.seterr(all='ignore')
        self.models.append(self.model.fit(X))

    # Run the model on input data : try and see how our model works
    def get_score(self, input_data):
        return self.model.score(input_data)

In [7]:
if __name__=='__main__':
    args = build_arg_parser().parse_args()
    input_folder = argparse.input_folder

    hmm_models = [] # initiate a variable to hold all the hmm models

    # parse the input dir
    for dirname in os.listdir(input_folder):
        # get the name/link of the subfolder
        subfolder = os.path.join(input_folder, dirname)
        if not os.path.isdir(subfolder): # if not a dir, pass/skip it
            continue

        # extract the label : another important concept
        label = subfolder[subfolder.rfind('/') + 1:]

        # initialize variables
        X = np.array([])
        y_words = []

        # Iterate through audio files leaving 1 file for testing in each class
        for filename in [x for x in os.listdir(subfolder) if x.endswith('.wav')][:-1]:
            # read the input file and extract the sampling freq and audio_data
            filepath = os.path.join(subfolder, filename)
            sampling_frequency, audio = wavfile.read(filepath)

            # extract the mfcc features
            mfcc_features = mfcc(audio, sampling_frequency,nfft=1200)

            # append to the X variable
            if len(X) == 0:
                X = mfcc_features
            else:
                X = np.append(X, mfcc_features, axis=0)

            # Append the label
            y_words.append(label)
        print('X.shape = ', X.shape)

        # Train and Save HMM model
        hmm_trainer = HMMTrainer()
        hmm_trainer.train(X)
        hmm_models.append((hmm_trainer, label))
        hmm_trainer = None

    # test files
    input_files = [
        #links to test files
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_ac_0.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_ac_1.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_ac_2.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_ac_3.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_bilgileri_0.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_bilgileri_1.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_bilgileri_2.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_bilgileri_3.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_kapat_0.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_kapat_1.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_kapat_2.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/demo_fider_kapat_3.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/gsm_durumu_0.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/gsm_durumu_1.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/gsm_durumu_2.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/gsm_durumu_3.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/nem_durumu_0.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/nem_durumu_1.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/nem_durumu_2.wav"
        "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands/nem_durumu_3.wav"
    ]

    # classify the input data
    for input_file in input_files:
        # read & extract from the input files
        sampling_frequency, audio_data = wavfile.read(input_file)

        # extract mfcc features
        mfcc_features = mfcc(audio, sampling_frequency, nfft=1200)

        # define variabes
        max_score = None
        output_label = None

        # iterate through all HMM Models and pick the one with the highest score
        for item in hmm_models:
            hmm_model, label = item 
            score = hmm_model.get_score(mfcc_features)
            if score > max_score:
                max_score = score 
                output_label = label

        # Print the output
        print("\nTrue:", input_file[input_file.find('/')+1:input_file.rfind('/')])
        print("Predicted:", output_label)

usage: ipykernel_launcher.py [-h] --input-folder INPUT_FOLDER
ipykernel_launcher.py: error: the following arguments are required: --input-folder


SystemExit: 2

/home/ix502iv/miniconda3/envs/TensorFlow/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
# test some input_file function

input_files = [
        "/home/ix502iv/Documents/Audio_Trad/HMM/Python/Day_5/data/audio/apple/apple15.wav",
        "/home/ix502iv/Documents/Audio_Trad/HMM/Python/Day_5/data/audio/banana/banana15.wav"
]
input_files

['/home/ix502iv/Documents/Audio_Trad/HMM/Python/Day_5/data/audio/apple/apple15.wav',
 '/home/ix502iv/Documents/Audio_Trad/HMM/Python/Day_5/data/audio/banana/banana15.wav']

In [17]:
for input_file in input_files:
    print(input_file[input_file.rfind('/')+1:input_file.rfind('.')])

apple15
banana15


### Error Handling:


- The error handling block has been added after the for loop, and it catches any exceptions that might occur during processing of an input file. If an exception is caught, the program prints an error message along with the filename and the error message. This ensures that the program does not crash if an exception occurs during processing of an input file.

In [ ]:
# function to parse input arguments
def build_arg_parser():
    parser = argparse.ArgumentParser(
        description="Trains the HMM Classifier")
    parser.add_argument(
        "--input-folder",
        dest="input_folder",
        required=True,
        help="Input folder containing the audio files in subfolders")
    return parser

if __name__=='__main__':
    try:
        args = build_arg_parser().parse_args()
        input_folder = args.input_folder
    except argparse.ArgumentError as e:
        print(f"Error: {str(e)}")
        sys.exit(1)

    hmm_models = [] # initiate a variable to hold all the hmm models

    # parse the input dir
    try:
        for dirname in os.listdir(input_folder):
            # get the name/link of the subfolder
            subfolder = os.path.join(input_folder, dirname)
            if not os.path.isdir(subfolder): # if not a dir, pass/skip it
                continue

            # extract the label : another important concept
            label = subfolder[subfolder.rfind('/') + 1:]

            # initialize variables
            X = np.array([])
            y_words = []

            # Iterate through audio files leaving 1 file for testing in each class
            for filename in [x for x in os.listdir(subfolder) if x.endswith('.wav')][:-1]:
                # read the input file and extract the sampling freq and audio_data
                filepath = os.path.join(subfolder, filename)
                try:
                    sampling_frequency, audio = wavfile.read(filepath)
                except ValueError as e:
                    print(f"Error reading file {filepath}: {str(e)}")
                    continue

                # extract the mfcc features
                mfcc_features = mfcc(audio, sampling_frequency,nfft=512)

                # append to the X variable
                if len(X) == 0:
                    X = mfcc_features
                else:
                    X = np.append(X, mfcc_features, axis=0)

                # Append the label
                y_words.append(label)

            # Train and Save HMM model
            hmm_trainer = HMMTrainer()
            hmm_trainer.train(X)
            hmm_models.append((hmm_trainer, label))
            hmm_trainer = None
    except OSError as e:
        print(f"Error accessing directory {input_folder}: {str(e)}")
        sys.exit(1)

    # test files
    input_files = [
        "/home/ix502iv/Documents/Audio_Trad/HMM/Python/Day_5/data/audio/apple/apple.wav",
        "/home/ix502iv/Documents/Audio_Trad/HMM/Python/Day_5/data/audio/banana/banana.wav"
    ]

    # classify the input data
    for input_file in input_files:
        try:
            # read & extract from the input files
            sampling_frequency, audio_data = wavfile.read(input_file)

            # extract mfcc features
            mfcc_features = mfcc(audio_data, sampling_frequency, nfft=512)

            # define variabes
            max_score = 0.0 # initially None
            output_label = "unknown"

            # iterate through all HMM Models and pick the one with the highest score
            for item in hmm_models:
                hmm_model, label = item 
                score = hmm_model.get_score(mfcc_features)
                if score > max_score:
                    max_score = score 
                    output_label = label

            # Print the output
            print("\nTrue:", input_file[input_file.rfind('/')+1:input_file.rfind('.')])
            print("Predicted:", output_label)

        except Exception as e:
            print(f"Error processing {input_file}: {str(e)}")


### Normalization or scaling of audio data.
- One common approach is to normalize the audio data by dividing it by the maximum absolute value. This will ensure that the data is scaled to a range between -1 and 1, which can help prevent issues with numerical precision and can also help to reduce the impact of background noise on the accuracy of the model.

- Apply normalization to the audio data before extracting MFCC features

In [ ]:
# define a function to normalize audio data
def normalize_audio(audio):
    # get the maximum absolute value
    max_abs = np.max(np.abs(audio))
    # divide by max_abs to normalize the audio data
    return audio / max_abs

# iterate through audio files leaving 1 file for testing in each class
for filename in [x for x in os.listdir(subfolder) if x.endswith('.wav')][:-1]:
    # read the input file and extract the sampling freq and audio_data
    filepath = os.path.join(subfolder, filename)
    sampling_frequency, audio = wavfile.read(filepath)

    # normalize the audio data
    audio = normalize_audio(audio)

    # extract the mfcc features
    mfcc_features = mfcc(audio, sampling_frequency,nfft=512)

    # append to the X variable
    if len(X) == 0:
        X = mfcc_features
    else:
        X = np.append(X, mfcc_features, axis=0)

    # Append the label
    y_words.append(label)
